In [1]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import wave
import pandas as pd
import soundfile as sf
import librosa.display
import IPython.display as ipd
from PIL import Image
import os
import scipy.signal as signal
from tqdm import tqdm

In [2]:
Df_A=pd.read_csv('/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/dataframes/Heart_Audio_Default_Folders_A.csv')

In [3]:
Df_A

,File,Class,Folder
0,201012172012.wav,Artifact,Atraining_artifact
1,201105040918.wav,Artifact,Atraining_artifact
2,201105041959.wav,Artifact,Atraining_artifact
3,201105051017.wav,Artifact,Atraining_artifact
4,201105060108.wav,Artifact,Atraining_artifact
...,...,...,...
119,201103200218.wav,Extrasystole,Atraining_extrahls
120,201104021355.wav,Extrasystole,Atraining_extrahls
121,201104140118.wav,Extrasystole,Atraining_extrahls
122,201104270458.wav,Extrasystole,Atraining_extrahls


In [4]:
sampling_rate = 44100
cutoff_freq = 195
duration = 3
pitch_shift_value = -0.5
time_shift_value = 1.0

In [5]:
# Step 1: Denoising using a low pass filter
def apply_low_pass_filter(audio, sampling_rate, cutoff_freq):
    nyquist_freq = 0.5 * sampling_rate
    normalized_cutoff_freq = cutoff_freq / nyquist_freq
    b, a = signal.butter(4, normalized_cutoff_freq, btype='low', analog=False)
    denoised_audio = signal.lfilter(b, a, audio)
    return denoised_audio

In [6]:
# Downsampling audio
def downsample_audio(audio,original_sampling_rate,target_sampling_rate):
    resampled_audio = librosa.resample(audio, orig_sr=original_sampling_rate, target_sr=target_sampling_rate)
    return resampled_audio

In [7]:
# Split audio into fixed-length segments
def split_audio(audio, segment_length):
    num_segments = len(audio) // segment_length
    segments = [audio[i*segment_length:(i+1)*segment_length] for i in range(num_segments)]
    return segments

In [8]:
# Apply time shifting to audio
def apply_time_shift(audio, shift_factor):
    shifted_audio = np.roll(audio, shift_factor)
    return shifted_audio

In [9]:
# Apply pitch shifting to audio
def apply_pitch_shift(audio, sampling_rate, pitch_shift_steps):
    pitch_shifted_audio = librosa.effects.pitch_shift(audio, sr=sampling_rate, n_steps=pitch_shift_steps)
    return pitch_shifted_audio

In [10]:
audio_dataset_path='/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/A_data'
for index_num,row in tqdm(Df_A.iterrows()):
    audio_path = os.path.join(os.path.abspath(audio_dataset_path),str(row["Folder"])+'/',str(row["File"]))
    audio, sampling_rate = librosa.load(audio_path, sr=None)

    # Denoising
    cutoff_frequency = 195
    denoised_audio = apply_low_pass_filter(audio, sampling_rate, cutoff_frequency)

    # Downsampling
    target_sampling_rate = sampling_rate // 10
    downsampled_audio = downsample_audio(denoised_audio, sampling_rate, target_sampling_rate)

    # Splitting audio
    segment_length = target_sampling_rate * 3
    segments = split_audio(downsampled_audio, segment_length)

    # Saving augmented audio segments
    output_folder = '/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/normalised_audio/'
    for i, segment in enumerate(segments):
        output_path = output_folder + '{}_segment_{}.wav'.format(str(row["File"])[:-4],i)
        sf.write(output_path, segment, target_sampling_rate)

124it [00:02, 58.24it/s]


In [11]:
folder_path = "/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/normalised_audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    time_shift_factor = 1
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    time_shifted = apply_time_shift(audio, time_shift_factor)
    output_folder = "/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/time_shifted_audio/"
    output_path = output_folder + '{}_time.wav'.format(str(file_name)[:-4])
    sf.write(output_path, time_shifted, target_sampling_rate)

In [12]:
folder_path = "/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/normalised_audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    pitch_shift_steps = -0.5
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    pitch_shifted = apply_pitch_shift(audio, target_sampling_rate, pitch_shift_steps)
    output_folder = "/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/pitch_shifted_audio/"
    output_path = output_folder + '{}_pitch.wav'.format(str(file_name)[:-4])
    sf.write(output_path, pitch_shifted, target_sampling_rate)

In [13]:
def save_spect(data, sr, filename):
    # Extract spectrogram
    spectrogram = librosa.feature.melspectrogram(y=data, sr=sr)
    # Convert to decibels
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
    # Plot spectrogram
    plt.figure(figsize=(1.28,1.28))
    librosa.display.specshow(spectrogram_db, sr=sr)
    plt.savefig(("/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/Aug_Audio_Spect_New/{}_spect.png").format(filename), transparent=True)
    plt.close()

In [14]:
folder_path = "/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/time_shifted_audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    save_spect(audio, sampling_rate, str(file_name)[:-4])

In [15]:
folder_path = "/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/pitch_shifted_audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    save_spect(audio, sampling_rate, str(file_name)[:-4])

In [16]:
def save_spect1(data, sr, filename):
    # Extract spectrogram
    spectrogram = librosa.feature.melspectrogram(y=data, sr=sr)

    # Convert to decibels
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    # Plot spectrogram
    plt.figure(figsize=(30,10))
    librosa.display.specshow(spectrogram_db, sr=sr)
    plt.savefig(("/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/Norm_Audio_Spect/{}_spect.png").format(filename), transparent=True)
    plt.close()
    img = Image.open(("/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/Norm_Audio_Spect/{}_spect.png").format(filename))
    img_resized = img.resize((128,128))

    #Save the resized image
    img_resized.save(("/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/Norm_Audio_Spect/{}_spect.png").format(filename))

In [17]:
folder_path = "/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/normalised_audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    save_spect1(audio, sampling_rate, str(file_name)[:-4])

In [18]:
a="201012172012_segment_0_pitch_spect.png"
b=Df_A.loc[Df_A["File"]==(a[:12]+".wav"),"Class"].item()
print(b)

Artifact


In [19]:
def time_warp(spec, W=5):
    num_bins, num_frames = spec.shape
    spec_aug = np.copy(spec)

    # Generate random control points for time warping
    control_points = [(0, 0), (num_bins-1, num_frames-1)]
    for _ in range(W):
        random_point = np.random.randint(1, num_bins-1)
        control_points.append((random_point, np.random.randint(0, num_frames-1)))

    # Apply time warping
    for i in range(len(control_points)-1):
        c1, c2 = control_points[i], control_points[i+1]
        spec_aug[c1[0]:c2[0]+1, :] = np.roll(spec[c1[0]:c2[0]+1, :], c2[1] - c1[1], axis=1)

    return spec_aug

In [20]:
import random

def save_augmented_spect(data, sr, filename):
    # Extract original spectrogram
    spectrogram = librosa.feature.melspectrogram(y=data, sr=sr)

    # Convert to decibels
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
    
    # Normalize the spectrogram to values between 0 and 1
    spectrogram_normalized = (spectrogram_db - np.min(spectrogram_db)) / (np.max(spectrogram_db) - np.min(spectrogram_db))

    # Scale the spectrogram to values between 1 and 255
    spectrogram_scaled = spectrogram_normalized * 255

    # Plot original spectrogram
    plt.figure(figsize=(1.28, 1.28))
    librosa.display.specshow(spectrogram_scaled, sr=sr)
    plt.savefig(("/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/Aug_Audio_Spect_New/{}_original.png").format(filename), transparent=True)
    plt.close()

    # Time-wrapped spectrogram
    time_wrapped_spectrogram = time_warp(spectrogram_scaled)
    plt.figure(figsize=(1.28, 1.28))
    librosa.display.specshow(time_wrapped_spectrogram, sr=sr)
    plt.savefig(("/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/Aug_Audio_Spect_New/{}_time_wrapped.png").format(filename), transparent=True)
    plt.close()

    # Frequency-masked spectrograms
    for i in range(2):
        frequency_masked_spectrogram = spectrogram.copy()
        start_freq = random.randint(0, spectrogram.shape[0] // 2)
        end_freq = random.randint(start_freq, spectrogram.shape[0] - 1)
        frequency_masked_spectrogram[start_freq:end_freq + 1, :] = 0
        frequency_masked_spectrogram_db = librosa.power_to_db(frequency_masked_spectrogram, ref=np.max)
        plt.figure(figsize=(1.28, 1.28))
        librosa.display.specshow(frequency_masked_spectrogram_db, sr=sr)
        plt.savefig(("/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/Aug_Audio_Spect_New/{}_freq_masked_{}.png").format(filename, i+1), transparent=True)
        plt.close()

In [21]:
folder_path = "/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/normalised_audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

for file_name in file_names:
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    save_augmented_spect(audio, sampling_rate, str(file_name)[:-4])

In [22]:
Spect_Dataset=[]
folder_path = "/Users/atrix/Documents/desktop_folders/PS/heartbeat_classification/Datasets/Aug_Audio_Spect_New"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    Spect_class=Df_A.loc[Df_A["File"]==(str(file_name[:12])+".wav"),"Class"].iloc[0]
    Spect_Dataset.append([file_name,Spect_class])

In [23]:
Spect_Df=pd.DataFrame(Spect_Dataset, columns=['Spect_File', 'Spect_Class'])

In [24]:
Spect_Df

,Spect_File,Spect_Class
0,201106070949_segment_0_time_spect.png,Artifact
1,201106211041_segment_0_freq_masked_1.png,Artifact
2,201108222253_segment_1_time_wrapped.png,Murmur
3,201106221254_segment_1_freq_masked_1.png,Artifact
4,201106221418_segment_1_time_wrapped.png,Normal
...,...,...
1741,201106161019_segment_2_time_wrapped.png,Artifact
1742,201101241433_segment_1_pitch_spect.png,Extrasystole
1743,201108222227_segment_0_time_spect.png,Murmur
1744,201106030612_segment_0_pitch_spect.png,Artifact


In [25]:
Spect_Df.to_csv('Heart_Spectrograms_Filenames.csv')